In [9]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import datetime
from citipy import citipy
from scipy import stats
from scipy.stats import linregress
from pprint import pprint
import gmaps
import os

#import api keys
from api_keys import gkey

In [10]:
#Readfile
cities_data_df = pd.read_csv("../weatherpy/weather_data.csv")
cities_data_df.dropna()
cities_data_df.head(3)

,Unnamed: 0,city,country,latitude,longitude,max_temperature,humidity,cloudiness,wind_speed
0,0,Cape town,ZA,-33.93,18.42,283.15,100.0,0.0,1.0
1,1,Port elizabeth,ZA,-33.92,25.57,285.15,87.0,0.0,2.6
2,2,Busselton,AU,-33.65,115.33,285.37,90.0,0.0,2.3


In [11]:
cities_data_df.dtypes

Unnamed: 0           int64
city                object
country             object
latitude           float64
longitude          float64
max_temperature    float64
humidity           float64
cloudiness         float64
wind_speed         float64
dtype: object

In [14]:
cities_data_df.count()

Unnamed: 0         576
city               576
country            573
latitude           576
longitude          576
max_temperature    576
humidity           576
cloudiness         576
wind_speed         576
dtype: int64

In [6]:
#configure gmaps
gmaps.configure(api_key=gkey)

#set variables
locations = cities_data_df[["latitude", "longitude"]]
humidity = cities_data_df["humidity"]

#Heatmap - humidity
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=10, point_radius=2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
#Narrow down the DataFrame to find your ideal weather condition.

narrowed_city_df = cities_data_df.loc[cities_data_df["max_temperature"] < 90]
narrowed_city_df = cities_data_df.loc[cities_data_df["max_temperature"] >= 70]
narrowed_city_df = cities_data_df.loc[cities_data_df["wind_speed"] < 15]
narrowed_city_df = cities_data_df.loc[cities_data_df["cloudiness"] >= 70]

#check if there is columns witn no information to drop
narrowed_city_df.count()

Unnamed: 0         244
city               244
country            244
latitude           244
longitude          244
max_temperature    244
humidity           244
cloudiness         244
wind_speed         244
dtype: int64

In [15]:
narrowed_city_df.head()

,Unnamed: 0,city,country,latitude,longitude,max_temperature,humidity,cloudiness,wind_speed
3,3,Neryungri,RU,56.67,124.65,293.18,82.0,100.0,0.85
5,5,College,US,64.86,-147.80,299.15,44.0,75.0,4.92
6,6,Geraldton,AU,-28.77,114.60,284.15,93.0,93.0,2.60
7,7,Avarua,CK,-21.21,-159.78,295.15,53.0,100.0,3.10
9,9,Puerto del rosario,ES,28.50,-13.86,296.15,78.0,75.0,7.20


In [16]:
#HOTEL
hotel_df = pd.DataFrame(narrowed_city_df)
hotel_df["hotel"] = ""


In [17]:
print(hotel_df)

Unnamed: 0                city country  latitude  longitude  \
3             3           Neryungri      RU     56.67     124.65   
5             5             College      US     64.86    -147.80   
6             6           Geraldton      AU    -28.77     114.60   
7             7              Avarua      CK    -21.21    -159.78   
9             9  Puerto del rosario      ES     28.50     -13.86   
..          ...                 ...     ...       ...        ...   
561         561              Mehamn      NO     71.04      27.85   
563         563            Were ilu      ET     10.60      39.43   
565         565                 Obo      CF      5.40      26.49   
566         566           Pontianak      ID     -0.03     109.33   
572         572             Lerwick      GB     60.15      -1.15   

     max_temperature  humidity  cloudiness  wind_speed hotel  
3             293.18      82.0       100.0        0.85        
5             299.15      44.0        75.0        4.92        

In [ ]:
# set up a parameters dictionary
params = {
    "radius": 5000,
    "type": "hotel",
    "key": gkey
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [ ]:
for index, row in hotel_df.iterrows():

    response = requests.get(base_url, params=params).json()
    
    results = response['results']

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["latitude", "longitude"]]